### Steps:
- Step 1. Go to https://namsor.app/api-documentation/ to register for an account and copy the api keys.
- Step 2. Replace the API_KEY with your api_keys

In [1]:
import csv
import requests, json
from tqdm import tqdm

'\nSteps:\nStep 1. Go to https://namsor.app/api-documentation/ to register for an account and copy the api keys.\nStep 2. Replace the API_KEY with your api_keys\n'

In [2]:
# Things you have to change
API_KEY = "YOUR_API_KEY" # replace with your api keys


# You can just input the last name
def get_region(last_name, first_name=""): 
    payload = {  # Initial settings
        "personalNames": [{
            "firstName": first_name,
            "lastName": last_name
        }]
    }
    headers = {
        "X-API-KEY": API_KEY, # API Key
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    response = requests.request("POST", "https://v2.namsor.com/NamSorAPIv2/api2/json/originBatch", json=payload, headers=headers)
    region_origin = json.loads(response.text)['personalNames'][0]['regionOrigin']
    return region_origin, response.text


In [ ]:
# Read files and extract names
name_lst = []
with open('NMS_citations.txt') as f:
    for line in f:
        line = line.rstrip()
        if len(line) < 1: continue
        
        text = line.split('(')[0]
        if '&' in text:
            for t in text.split('&'):
                t = t.strip()
                if t[-1] == ',': t = t[:-1] # remove additional comma
                name_lst.append(t)
        else:
            name_lst.append(text.strip())

    
# Get region from namsor
ind_name_lst = set(name_lst)
# for i in ind_name_lst:
#     print(i)

try:
    ind_name_lst.remove("")
except ValueError:
    print("Item not found in the list.")

region_lst, region_txt_lst = [], []
for n in tqdm(ind_name_lst):
    lst_name = n.split(',')[0]
    reg, reg_txt = get_region(lst_name)
    region_lst.append(reg)
    region_txt_lst.append(reg_txt)
    


# Write the data into csv files
data = []
for i, j, k in zip(ind_name_lst, region_lst, region_txt_lst):
    data.append({"name": i, "region": j, "text": k})
    
# Specify the CSV file name
with open('NMS_name_dict.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['name', 'region', 'text'])
    writer.writeheader()

    for row in data:
        writer.writerow(row)

print("CSV file has been created successfully.")